In [19]:
import scipy as sc
import numpy as np
#!pip install gpxpy
import gpxpy
from geopy import distance
import json
import pandas as pd


# Putting it all together:

1. read gpx file(s) into an array with elevations and coordinates
2. read segement data into array with elevations and coordinates
3. get total distance and total elevation difference from segment
4. sweep through gpx arrays to find sections that roughly fit the segment
5. calculate elapsed times for those sections
6. use DTW to see if segments are actually similar
    - if they are, compare times.

- Thought - skip the prelim data sweep and just use DTW

In [2]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [ ]:
from geopy import distance
# method to calculate distances between point
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).m
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance

In [14]:
# method to make GPX file into array of elevations, times, distances
def file_to_elevation_arr(fileName):
        gpx_file = open(fileName, 'r')
        gpx = gpxpy.parse(gpx_file)
        elevevation_time_and_distance = []
        pointOne = None
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    pointTwo = point
                    distance = 0
                    if pointOne:
                        distance = distanceBetweenPoints(pointOne, pointTwo)
                    #print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))
                    elevevation_time_and_distance.append((point.elevation,point.time, distance))
        return elevevation_time_and_distance

In [5]:
fileOne = "data/Walt's Strava/activities/4844961125.gpx"
fileTwo  = "data/Walt's Strava/activities/4812864781.gpx"

In [15]:
arrOne = file_to_elevation_arr(fileOne)
arrTwo = file_to_elevation_arr(fileTwo)

In [7]:
print(dtw(arrOne, arrTwo,10))

[[0.00000e+00         inf         inf ...         inf         inf
          inf]
 [        inf 3.00000e-01 6.00000e-01 ...         inf         inf
          inf]
 [        inf 5.00000e-01 5.00000e-01 ...         inf         inf
          inf]
 ...
 [        inf         inf         inf ... 3.73075e+04 3.73102e+04
  3.73129e+04]
 [        inf         inf         inf ...         inf 3.73102e+04
  3.73129e+04]
 [        inf         inf         inf ...         inf         inf
  3.73127e+04]]


In [17]:
segment = "data/Elevation JSON/segment_location_data_elevation_0.json"

In [35]:
df = pd.read_json(segment)
df.head()


,lat,lon,elev_meters,distance
0,34.09464,-117.71377,355.14,0.000000
1,34.09467,-117.71362,355.22,14.236931
2,34.09467,-117.71327,355.12,32.298915
3,34.09476,-117.71181,356.33,135.106672
4,34.09474,-117.71131,356.34,46.194346


In [49]:
def dataframe_segment_to_list(df):
    sg =[]
    # Iterate over each row
    for index, rows in df.iterrows():
        # Create list for the current row
        my_list =[rows[2], rows[3]]
        
        # append the list to the final list
        sg.append(my_list)
    return sg
  

In [47]:
#print(segment_data)
def total_segment_elevation(segment_data):
    total_elev = segment_data[len(segment_data)-1][0] - segment_data[0][0]
    return total_elev

def total_segment_distance(segment_data):
    total_distance = 0
    for i in segment_data:
        total_distance += i[1]
    return total_distance

In [50]:
def sweep_user_activity(user_data, segement_data):
    list_of_matches = []
    seg_distance = total_segment_distance(segement_data)
    seg_elevation = total_segment_elevation(segement_data)
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    count = 0
    while user_distance < seg_distance and count < len(user_distance):
        # until we get to the window 
        user_elev = user_data[count][0]-user_start_elev
        user_distance += user_data[count][2]
        count += 1
    delta_distance = user_distance - user_data[0][2]
    # now we have the window, so we need to iterate through and see if the profiles match
    count2 = 0
    user_elev_two = user_data[0][0]
    while count < len(user_distance):
        if user_elev_two - user_elev in range(seg_distance-100, seg_distance+100, .1):
            list_of_matches.append(user_data[count:count2])
            # add match to list
    return list_of_matches




In [51]:
# TODO: add DTW algorithm match